In [377]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint
# import librosa
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils

from matplotlib import pyplot as plt
# import tensorflow as tf

In [401]:
file_array = os.listdir("/home/rohan/BE_project/raw/enroll");

directory = os.path.join(os.getcwd(), 'raw/enroll')

for file_name in file_array :
#/home/punit/Downloads/test/{}  directory which contains raw files 
#/home/punit/Downloads/processed/{} directory which contains processed files
#install sox before the script is run. 
#Meaning of parameters refer this article https://digitalcardboard.com/blog/2009/08/25/the-sox-of-silence/
#     os.system("rm -f /home/rohan/BE_project/raw/enroll_processed/*")
# 0.3 -70d -1 0.1 1% - 57
#     cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/enroll/noise-{} trim 0 0.900".format(file_name, file_name)
#     os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll/noise-{} -n noiseprof /home/rohan/BE_project/raw/enroll/noise-{}.prof".format(file_name, file_name)
#     os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/temp/{} rate 44100".format(file_name, file_name, file_name)
#     os.system(cmd);

    (rate,sig) = wav.read(os.path.join(directory, file_name))
    if int(rate) > 8000:
        cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/enroll_processed/{} silence 1 0.3 -45d -1 0.1 1% lowpass 7000".format(file_name, file_name)
        os.system(cmd);
    else:
        cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/enroll_processed/{} silence 1 0.3 -55d -1 0.1 1% lowpass 3500".format(file_name, file_name)
        os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll_processed/{} /home/rohan/BE_project/raw/enroll_processed/{} lowpass 20000".format(file_name, file_name)
        
    

In [402]:
# file_array = os.listdir("/home/rohan/BE_project/raw/test");
# for file_name in file_array :
# #/home/punit/Downloads/test/{}  directory which contains raw files 
# #/home/punit/Downloads/processed/{} directory which contains processed files
# #install sox before the script is run. 
# #Meaning of parameters refer this article https://digitalcardboard.com/blog/2009/08/25/the-sox-of-silence/
# #     os.system("rm -f /home/rohan/BE_project/raw/test_processed/*")
#     cmd = "sox /home/rohan/BE_project/raw/test/{} /home/rohan/BE_project/raw/test_processed/{} silence 1 0.3 -45d -1 0.1 1% lowpass 7000".format(file_name, file_name)
# #     print(cmd);
#     os.system(cmd);

    
file_array = os.listdir("/home/rohan/BE_project/raw/test");

directory = os.path.join(os.getcwd(), 'raw/test')

for file_name in file_array :
#/home/punit/Downloads/test/{}  directory which contains raw files 
#/home/punit/Downloads/processed/{} directory which contains processed files
#install sox before the script is run. 
#Meaning of parameters refer this article https://digitalcardboard.com/blog/2009/08/25/the-sox-of-silence/
#     os.system("rm -f /home/rohan/BE_project/raw/enroll_processed/*")
# 0.3 -70d -1 0.1 1% - 57
#     cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/enroll/noise-{} trim 0 0.900".format(file_name, file_name)
#     os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll/noise-{} -n noiseprof /home/rohan/BE_project/raw/enroll/noise-{}.prof".format(file_name, file_name)
#     os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/temp/{} rate 44100".format(file_name, file_name, file_name)
#     os.system(cmd);

    (rate,sig) = wav.read(os.path.join(directory, file_name))
    if int(rate) > 8000:
        cmd = "sox /home/rohan/BE_project/raw/test/{} /home/rohan/BE_project/raw/test_processed/{} silence 1 0.3 -45d -1 0.1 1% lowpass 7000".format(file_name, file_name)
        os.system(cmd);
    else:
        cmd = "sox /home/rohan/BE_project/raw/test/{} /home/rohan/BE_project/raw/test_processed/{} silence 1 0.3 -55d -1 0.1 1% lowpass 3500".format(file_name, file_name)
        os.system(cmd);
#     cmd = "sox /home/rohan/BE_project/raw/enroll_processed/{} /home/rohan/BE_project/raw/enroll_processed/{} lowpass 20000".format(file_name, file_name)
        
    

In [409]:
# every frame is considered independent

no_of_features = 13
no_of_fbank_features = 13
no_of_columns = (3 * no_of_features) + no_of_fbank_features
test_frames = 50
classes = 10

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
        directory = os.path.join(os.getcwd(), 'raw/enroll_processed')
        no_of_frames = 800
        start_frame = 10
    elif (dataset_type == "test"):    
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
        directory = os.path.join(os.getcwd(), 'raw/test_processed')
        no_of_frames = test_frames
        start_frame = 1
        
    dataset = numpy.empty([0, no_of_columns + 1])
    
    for file in os.listdir(directory):
        
        # filter speakers
        n = int(classes/2)
#         print(n)
        names = []
        
        for i in range (1, n+1):
            names.append('F0' + '%02d'%(i))
            names.append('M0' + '%02d'%(i))

        if any(name in file for name in names):
            
            # extract mfcc vectors
#             print(file)
            (rate,sig) = wav.read(os.path.join(directory, file))
            fbank_feat = logfbank(sig,rate)
            mfcc_feat = mfcc(sig,rate,winlen=0.032,winstep=0.016,numcep=13,nfft=2048)
            
            print("SR :" + str(rate) + " " + file)
#             y, sr = librosa.load(os.path.join(directory, file))
#             mfcc_feat = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=2048).T
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
#             fbank_feat = logfbank(sig,rate)
            mfcc_vectors = mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            dmfcc_vectors = d_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            ddmfcc_vectors = dd_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            fbank_vectors = fbank_feat[start_frame:start_frame+no_of_frames,:no_of_fbank_features]
            
            feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors, fbank_vectors))
#             print(feature_vectors.shape)
            
            # get speaker index from filename
            speaker_index = file.split("_")[0]
            if speaker_index[0] == 'M':
                speaker_index = n + int(speaker_index[3:])
            else:
                speaker_index = int(speaker_index[3:])

            #append speaker index to feature vectors
            np_speaker_index = numpy.array([speaker_index])
            temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
            concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             print(concatenated_feature_vector.shape)
#             print(fbank_vectors.shape)
            
            # append file's data to dataset
            dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            
    
    return dataset


In [410]:
from keras.models import Sequential
import numpy as numpy 

In [411]:
# from numpy import genfromtxt
my_data = get_feature_vectors("train")
numpy.random.shuffle(my_data)
# get_feature_vectors("test")

SR :44100 M003_punit.wav
SR :44100 F002_ritu.wav


SR :8000 M002_shubham.wav


SR :44100 M001_rohan.wav
SR :44100 F001_srishti.wav
SR :8000 M004_sudesh.wav
SR :8000 F004_aditi.wav
SR :8000 M005_abhinav.wav
SR :8000 F003_shraddha.wav


In [412]:
# numpy.set_printoptions(threshold=numpy.nan)
# print(my_data)
print(my_data.shape)

(7200, 53)


In [413]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(7200, 1)


In [414]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(7200, 52)
(1, 52)
(1, 52)
(7200, 52)


In [415]:
from keras import utils as np_utils
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

one_hot_labels = np_utils.to_categorical(Y, num_classes=classes+1)
print(one_hot_labels)

[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [416]:
def train():

    # MultiLayer Perceptron
    model = Sequential()

    # 3000 0.6 1500 0.6 -> 57% acc
    # 2560 0.7 1280 0.7 -> cnn_model = cnn_train()
# test_cnn(cnn_model)60% acc
    # 2560 0.7 1600 0.7 -> 51% acc
    # 2560 0.6 1600 0.7 -> 49%
    # 2560 0.7 1280 -   -> 50
    # 3000 0.7 1280 0.7 -> 54% acc
    # 2560 0.7 1280 0.8 -> 45% acc

    model.add(Dense(3000, activation='tanh', input_dim=no_of_columns))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    
#     model.add(Dense(2560, activation='tanh', input_dim=no_of_columns))
#     model.add(Dropout(0.7))
#     model.add(Dense(1280, activation='tanh'))
#     model.add(Dropout(0.7))
#     65
    model.add(Dense(classes+1, activation='softmax'))

    sgd = SGD(lr=0.01, decay=3e-5, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=20, batch_size=32)
    
    return model
# score = model.evaluate(x_test, y_test, batch_size=128)

In [417]:
def test(model):
    
    test_model = get_feature_vectors("test")
    print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes+1)

#     print(model.test_on_batch(test_X, test_labels, sample_weight=None))
#     print(model.metrics_names)
    predictions = model.predict(normalized_test_X)
    
    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))
    
    for t, p in zip(test_Y[::test_frames].T[0], predicted_Y):
        print (int(t), p)
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    correct = sum(x == 0 for x in diff)
    total = len(predicted_Y)
    percent = correct/total * 100
    print(str(correct) + " / " + str(total) + " = " + str(percent))
    
    loss = str(numpy.sum(numpy.abs(diff)))
    print("Loss = " + loss)
    
    return correct, percent, loss

In [418]:
model = train()
test(model)

Epoch 1/20
7200/7200 [==============================] - 5s 713us/step - loss: 0.9210 - acc: 0.6700
Epoch 2/20
7200/7200 [==============================] - 5s 750us/step - loss: 0.5822 - acc: 0.7851
Epoch 3/20
7200/7200 [==============================] - 5s 743us/step - loss: 0.5025 - acc: 0.8197
Epoch 4/20
7200/7200 [==============================] - 8s 1ms/step - loss: 0.4510 - acc: 0.8386
Epoch 5/20
7200/7200 [==============================] - 6s 839us/step - loss: 0.4132 - acc: 0.8583
Epoch 6/20
7200/7200 [==============================] - 6s 860us/step - loss: 0.4002 - acc: 0.8621
Epoch 7/20
7200/7200 [==============================] - 6s 858us/step - loss: 0.3634 - acc: 0.8751
Epoch 8/20
7200/7200 [==============================] - 6s 858us/step - loss: 0.3510 - acc: 0.8760
Epoch 9/20
7200/7200 [==============================] - 6s 807us/step - loss: 0.3265 - acc: 0.8818
Epoch 10/20
7200/7200 [==============================] - 6s 830us/step - loss: 0.3253 - acc: 0.8892
Epoch 11/20

SR :44100 M001_rohan_6.wav
SR :44100 F002_ritu_5.wav
SR :8000 F004_aditi_2.wav


SR :8000 M004_sudesh_5.wav
SR :44100 F002_ritu_6.wav
SR :8000 F003_shraddha_2.wav
SR :8000 M002_shubham_8.wav


SR :8000 F003_shraddha_5.wav


SR :44100 F001_srishti_5.wav
SR :44100 F001_srishti_1.wav
SR :8000 M002_shubham_7.wav


...

SR :8000 M004_sudesh_2.wav
SR :44100 F001_srishti_4.wav
SR :44100 F002_ritu_2.wav


SR :8000 F003_shraddha_7.wav
SR :44100 M001_rohan_5.wav


SR :8000 M005_abhinav_4.wav
SR :44100 M001_rohan_4.wav
SR :8000 M004_sudesh_7.wav


SR :44100 M003_punit_4.wav
SR :44100 M003_punit_1.wav
(3300, 53)
6 6
2 1
4 4
9 7
2 2
3 7
7 9
3 3
1 1
1 2
7 7
10 10
6 6
10 10
9 10
1 1
1 1
3 3
4 4
1 1
9 7
4 4
4 4
2 1
6 6
2 1
9 10
10 10
7 7
3 3
7 7
2 2
10 10
6 6
3 4
6 6
4 4
8 8
7 10
7 9
3 4
7 7
4 4
1 1
4 4
9 7
8 8
3 3
6 6
1 1
10 10
4 4
9 10
10 10
10 7
7 7
9 7
1 1
2 1
3 3
6 6
10 10
6 6
9 9
8 8
8 8
47 / 66 = 71.2121212121
Loss = 32.0


(47, 71.212121212121218, '32.0')

In [419]:
def cnn_train():
    
    temp = normalized_X.reshape(normalized_X.shape[0], no_of_columns, 1)
    
    model = Sequential()
    # 13 7 1 1 0.25 60 0.25 10 - 70%
    
    model.add(Convolution1D(52, 13, activation='tanh', input_shape=(no_of_columns,1)))
    model.add(Convolution1D(52, 7, activation='tanh'))
    model.add(Convolution1D(13, 3, activation='tanh'))
    
    # stride = 2 - 70
    # 20, 10, 17 op - 64
    
    model.add(MaxPooling1D(pool_size=(1)))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1000, activation='tanh'))
    model.add(Dropout(0.25))
    
    # 0.4 70
    model.add(Dense(classes+1, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(temp, one_hot_labels, epochs=20, batch_size=100)
    return model

In [420]:
def test_cnn(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes+1)
    
    test_X = test_X.reshape(test_X.shape[0], no_of_columns, 1)
    normalized_test_X = normalized_test_X.reshape(normalized_test_X.shape[0], no_of_columns, 1)
    
    print(model.test_on_batch(normalized_test_X, test_labels, sample_weight=None))
    print(model.metrics_names)
    predictions = model.predict(normalized_test_X)

    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))

    # print(predicted_Y)
    # print(test_Y[::40].T)
    
    indices = numpy.argmax(predictions, axis=1)
    majority = []
    
    for i in range(0, len(indices), test_frames):
        majority.append(find_majority(indices[i:i + test_frames]))
        
#     majority = 
    for t, p, m in zip(test_Y[::test_frames].T[0], predicted_Y, majority):
        print(int(t), p, m[0])
    
#     for t, p in zip(test_Y.T[0], indices):
#         print(int(t), p)
    
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    maj_diff = numpy.array(majority)[:, 0] - test_Y[::test_frames].T[0]

    numerator = sum(x == 0 for x in diff)
    denominator = len(predicted_Y)
    
    numerator2 = sum(x == 0 for x in maj_diff)
    denominator2 = len(maj_diff)
    

    print("Accuracy prob_diff: {} of {} - {}".format(numerator, denominator, numerator/denominator))
    
    print("Accuracy maj_diff: {} of {} - {}".format(numerator2, denominator2, numerator2/denominator2))
    

In [421]:
cnn_model = cnn_train()
test_cnn(cnn_model)

Epoch 1/20
7200/7200 [==============================] - 11s 1ms/step - loss: 0.9205 - acc: 0.6793
Epoch 2/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.5152 - acc: 0.8040
Epoch 3/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4030 - acc: 0.8511
Epoch 4/20
7200/7200 [==============================] - 12s 2ms/step - loss: 0.3161 - acc: 0.8865
Epoch 5/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.2366 - acc: 0.9157
Epoch 6/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.1893 - acc: 0.9333
Epoch 7/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.1317 - acc: 0.9594
Epoch 8/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.1057 - acc: 0.9679
Epoch 9/20
7200/7200 [==============================] - 10s 1ms/step - loss: 0.0750 - acc: 0.9794
Epoch 10/20
7200/7200 [==============================] - 9s 1ms/step - loss: 0.0598 - acc: 0.9856
Epoch 11/20
7200/7200 [===

SR :44100 M001_rohan_6.wav
SR :44100 F002_ritu_5.wav
SR :8000 F004_aditi_2.wav


SR :8000 M004_sudesh_5.wav
SR :44100 F002_ritu_6.wav
SR :8000 F003_shraddha_2.wav


SR :8000 M002_shubham_8.wav
SR :8000 F003_shraddha_5.wav


SR :44100 F001_srishti_5.wav
SR :44100 F001_srishti_1.wav
SR :8000 M002_shubham_7.wav


...

SR :44100 M003_punit_2.wav
SR :8000 F003_shraddha_8.wav


SR :44100 M001_rohan_2.wav
SR :44100 F001_srishti_6.wav
SR :8000 M005_abhinav_2.wav
SR :8000 F004_aditi_4.wav
SR :8000 M004_sudesh_8.wav
SR :8000 M005_abhinav_5.wav
SR :8000 M005_abhinav_8.wav
SR :8000 M002_shubham_6.wav


SR :8000 M004_sudesh_2.wav
SR :44100 F001_srishti_4.wav
SR :44100 F002_ritu_2.wav


SR :8000 F003_shraddha_7.wav
SR :44100 M001_rohan_5.wav
SR :8000 M005_abhinav_4.wav


SR :44100 M001_rohan_4.wav
SR :8000 M004_sudesh_7.wav
SR :44100 M003_punit_4.wav
SR :44100 M003_punit_1.wav
[array(2.012251615524292, dtype=float32), array(0.6084848642349243, dtype=float32)]
['loss', 'acc']
6 6 6
2 1 1
4 4 4
9 9 9
2 2 2
3 7 7
7 9 9
3 3 3
1 1 1
1 2 2
7 7 7
10 10 10
6 6 6
10 10 10
9 9 9
1 1 1
1 1 1
3 4 4
4 4 4
1 1 1
9 7 7
4 4 4
4 4 4
2 1 1
6 6 6
2 1 1
9 9 9
10 10 10
7 7 7
3 3 3
7 7 7
2 1 1
10 10 10
6 6 6
3 4 4
6 6 6
4 4 4
8 8 8
7 10 7
7 7 7
3 4 4
7 7 7
4 4 4
1 1 1
4 4 4
9 9 9
8 8 8
3 4 4
6 6 6
1 1 1
10 10 10
4 4 4
9 10 10
10 10 10
10 10 10
7 7 7
9 7 7
1 1 1
2 1 1
3 3 3
6 6 6
10 10 10
6 6 6
9 9 9
8 8 8
8 8 8
Accuracy prob_diff: 50 of 66 - 0.7575757575757576
Accuracy maj_diff: 51 of 66 - 0.7727272727272727


In [422]:
def dense_cnn_train():
    
    model = Sequential()
    # 13 7 1 1 0.25 60 0.25 10 - 70%
    
    model.add(Dense(2560, activation='tanh', input_dim=no_of_columns))
    model.add(Dropout(0.2))
    model.add(Dense(1300, activation='tanh'))

    model.add(Reshape((52, -1)))
    model.add(Convolution1D(52, 13, activation='tanh'))
    model.add(Convolution1D(52, 7, activation='tanh'))
    model.add(Convolution1D(13, 3, activation='tanh'))
    
    # stride = 2 - 70
    # 20, 10, 17 op - 64
    
    model.add(MaxPooling1D(pool_size=(1)))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1000, activation='tanh'))
    model.add(Dropout(0.25))
    
    # 0.4 70
    model.add(Dense(classes+1, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=100)
    return model

In [423]:
def dense_cnn_test(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes+1)
    
#     test_X = test_X.reshape(test_X.shape[0], no_of_columns, 1)
#     normalized_test_X = normalized_test_X.reshape(normalized_test_X.shape[0], no_of_columns, 1)
    
    print(model.test_on_batch(normalized_test_X, test_labels, sample_weight=None))
    print(model.metrics_names)
    predictions = model.predict(normalized_test_X)

    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))

    # print(predicted_Y)
    # print(test_Y[::40].T)
    
    indices = numpy.argmax(predictions, axis=1)
    majority = []
    
    for i in range(0, len(indices), test_frames):
        majority.append(find_majority(indices[i:i + test_frames]))
        
#     majority = 
    for t, p, m in zip(test_Y[::test_frames].T[0], predicted_Y, majority):
        print(int(t), p, m[0])
    
#     for t, p in zip(test_Y.T[0], indices):
#         print(int(t), p)
    
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    maj_diff = numpy.array(majority)[:, 0] - test_Y[::test_frames].T[0]

    numerator = sum(x == 0 for x in diff)
    denominator = len(predicted_Y)
    
    numerator2 = sum(x == 0 for x in maj_diff)
    denominator2 = len(maj_diff)
    

    print("Accuracy prob_diff: {} of {} - {}".format(numerator, denominator, numerator/denominator))
    
    print("Accuracy maj_diff: {} of {} - {}".format(numerator2, denominator2, numerator2/denominator2))
    

# print(predicted_Y)
# print(test_Y[::40].T)

# for t, p in zip(test_Y[::40].T[0], predicted_Y):
#     print (int(t), p)



In [424]:
dense_cnn_model = dense_cnn_train()
dense_cnn_test(dense_cnn_model)


Epoch 1/10
7200/7200 [==============================] - 23s 3ms/step - loss: 0.6836 - acc: 0.7532
Epoch 2/10
7200/7200 [==============================] - 24s 3ms/step - loss: 0.3680 - acc: 0.8667
Epoch 3/10
7200/7200 [==============================] - 26s 4ms/step - loss: 0.2495 - acc: 0.9097
Epoch 4/10
7200/7200 [==============================] - 26s 4ms/step - loss: 0.1898 - acc: 0.9339
Epoch 5/10
7200/7200 [==============================] - 24s 3ms/step - loss: 0.1170 - acc: 0.9585
Epoch 6/10
7200/7200 [==============================] - 24s 3ms/step - loss: 0.0585 - acc: 0.9803
Epoch 7/10
7200/7200 [==============================] - 24s 3ms/step - loss: 0.0467 - acc: 0.9832
Epoch 8/10
7200/7200 [==============================] - 26s 4ms/step - loss: 0.0396 - acc: 0.9864
Epoch 9/10
7200/7200 [==============================] - 25s 3ms/step - loss: 0.0399 - acc: 0.9876
Epoch 10/10
7200/7200 [==============================] - 25s 3ms/step - loss: 0.0159 - acc: 0.9956


SR :44100 M001_rohan_6.wav
SR :44100 F002_ritu_5.wav
SR :8000 F004_aditi_2.wav


SR :8000 M004_sudesh_5.wav
SR :44100 F002_ritu_6.wav
SR :8000 F003_shraddha_2.wav


SR :8000 M002_shubham_8.wav
SR :8000 F003_shraddha_5.wav
SR :44100 F001_srishti_5.wav


SR :44100 F001_srishti_1.wav
SR :8000 M002_shubham_7.wav
SR :8000 M005_abhinav_1.wav
SR :44100 M001_rohan_3.wav


...

SR :8000 F003_shraddha_8.wav
SR :44100 M001_rohan_2.wav
SR :44100 F001_srishti_6.wav
SR :8000 M005_abhinav_2.wav
SR :8000 F004_aditi_4.wav
SR :8000 M004_sudesh_8.wav
SR :8000 M005_abhinav_5.wav


SR :8000 M005_abhinav_8.wav
SR :8000 M002_shubham_6.wav
SR :8000 M004_sudesh_2.wav


SR :44100 F001_srishti_4.wav
SR :44100 F002_ritu_2.wav
SR :8000 F003_shraddha_7.wav


SR :44100 M001_rohan_5.wav
SR :8000 M005_abhinav_4.wav
SR :44100 M001_rohan_4.wav
SR :8000 M004_sudesh_7.wav


SR :44100 M003_punit_4.wav
SR :44100 M003_punit_1.wav
[array(2.1971192359924316, dtype=float32), array(0.6187878847122192, dtype=float32)]
['loss', 'acc']
6 6 6
2 1 1
4 4 4
9 9 9
2 2 2
3 7 7
7 9 9
3 3 3
1 1 1
1 2 2
7 7 7
10 10 10
6 6 6
10 10 10
9 10 10
1 1 1
1 1 1
3 3 3
4 4 4
1 1 1
9 7 7
4 4 4
4 4 4
2 1 1
6 6 6
2 1 1
9 10 10
10 10 10
7 7 7
3 3 3
7 7 7
2 2 2
10 10 10
6 6 6
3 4 4
6 6 6
4 4 4
8 8 8
7 3 3
7 9 9
3 4 4
7 10 10
4 4 4
1 1 1
4 4 4
9 9 9
8 8 8
3 3 3
6 6 6
1 1 1
10 10 10
4 4 4
9 9 9
10 10 10
10 7 7
7 7 7
9 7 7
1 1 1
2 1 1
3 3 3
6 6 6
10 10 10
6 6 6
9 9 9
8 8 8
8 8 8
Accuracy prob_diff: 49 of 66 - 0.7424242424242424
Accuracy maj_diff: 49 of 66 - 0.7424242424242424


In [29]:
def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum

In [356]:
# try removing lowpass for lower sample rate
# w/o punit - 8 speakers
# 67 67 70 dense
# 80 74 74 cnn 
# 74 77 77 dense_cnn

# with punit - up
